In [1]:
import pandas as pd
import numpy as np
import re
import konlpy

okt = konlpy.tag.Okt()

C:\ProgramData\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [5]:
data = pd.read_csv("C:/Users/user/Downloads/190909/ratings_train.txt", sep="\t")
data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
text = data.loc[0,'document']
text

'아 더빙.. 진짜 짜증나네요 목소리'

In [7]:
# 1. 특수문자 제거
text = re.sub("[`!@#$%^&*()_+=[\]\-~{};:,./?]", "", text)
text


'아 더빙 진짜 짜증나네요 목소리'

In [8]:
# 2. Stemmer
okt.morphs(text, stem=True)

['아', '더빙', '진짜', '짜증나다', '목소리']

In [9]:
# 3. na제거
data = data.dropna()
data = data.reset_index(drop=True)

In [10]:
# 3. BOW
words = []
docs = []
for i in range(149995):
  text = data['document'][i]

  # 1. 특문제거
  text = re.sub("[`!@#$%^&*()_+=[\]\-~{};:,./?]", "", text)

  # 2. stem
  stems = okt.morphs(text, stem=True)

  # 3. bow
  words = words + stems

  # 4. 전처리된 문자열
  text_pre = " ".join([w for w in stems])

  docs.append(text_pre)
  

# 중복제거
voca = set(words)  

KeyError: 149995

In [12]:
len(docs)

149995

In [13]:
# text to vector
text

'한국 영화 최초로 수간하는 내용이 담긴 영화'

In [14]:
t = Tokenizer()
t.fit_on_texts(docs)

# text_to_seq
sequences = t.texts_to_sequences(docs)

# padding
max_len = max(len(s) for s in sequences)
x_train = pad_sequences(sequences, max_len)

In [15]:
y_train = data['label'][:150000].values

In [0]:
x_train

array([[  0,   0,   0, ...,  27, 272, 751],
       [  0,   0,   0, ..., 939, 775,  30],
       [  0,   0,   0, ..., 241,  21,  19],
       ...,
       [  0,   0,   0, ...,  90, 284, 229],
       [  0,   0,   0, ...,   8,  79,   2],
       [  0,   0,   0, ..., 432, 385, 152]], dtype=int32)

78

In [16]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils

In [26]:
model = Sequential()
model.add(Embedding(len(set(words)), 256, input_length = max_len))
model.add(LSTM(256))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=4000, epochs=3, validation_split=0.2)

Train on 119996 samples, validate on 29999 samples
Epoch 1/3
119996/119996 [==============================] - 308s 3ms/step - loss: 0.5828 - accuracy: 0.7093 - val_loss: 0.4299 - val_accuracy: 0.8125
Epoch 2/3
119996/119996 [==============================] - 308s 3ms/step - loss: 0.3604 - accuracy: 0.8498 - val_loss: 0.3573 - val_accuracy: 0.8462
Epoch 3/3
119996/119996 [==============================] - 307s 3ms/step - loss: 0.2965 - accuracy: 0.8796 - val_loss: 0.3571 - val_accuracy: 0.8488


In [27]:
from keras.callbacks import ModelCheckpoint

In [28]:
model_json = model.to_json()
with open("model.json", "w") as json_file : 
    json_file.write(model_json)

In [29]:
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [31]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(t, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
model.predict_classes(x_train[0:10])

array([[0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1]])